## ENPM 809F Final Project
## MQTT Protocol Simulation for Home Automation System
## Muhammad Adil Sohail & Waleed Waris
## 12/14/2020

* This project script will create an IoT simulation which will use MQTT protocol to simulate an IoT network in a home automation system. The simulation will leverage a dataset that provides actual data from sensors in a home automation system such as temperature, humidity, light on/off, etc. Data from the dataset will be read, published to a broker, and recieved by a smartphone application that is subscribed to the proper topic. In addition, commands for certain controllers can be sent from the smartphone application.

* This is one of the two modules for the project. The other being Client_publisher. This module will read the published messages from Client_publisher. A smartphone application will be subscribed to the respective topic on the server, recieve the sensor data, and print it. Also, controllers subscribed to the respective topic will receive commands sent from the mobile_devices. 

* A GUI will be leveraged to display the sensor data / actuator commands

## 1.0 Define MQTT Protocol Functions

In [1]:
# Import Neccesary Libraries 
import paho.mqtt.client as mqtt
import time
import pandas as pd
import os 
import copy
from tkinter import *
import time
import datetime

In [2]:
# Defines callback function to show when mobile_device has connected. Also, mobile_device subscribes to proper topic.

def on_connect_mobile_device(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("house/livingroom/time")
    client.subscribe("house/livingroom/CO")
    client.subscribe("house/livingroom/humidity")
    client.subscribe("house/livingroom/lightdetected")
    client.subscribe("house/livingroom/lpg")
    client.subscribe("house/livingroom/motiondetected")
    client.subscribe("house/livingroom/smoke")
    client.subscribe("house/livingroom/temp")  

In [3]:
# The following lines of code will store the published/subscribed messages in a database based on a topic
# data_store.pop(0)
def update_df(data_store):
    
    composite_list = [data_store[x:x+8] for x in range(0, len(data_store),8)] #Splits data by 8 entires 
    df_Smarthome = pd.DataFrame.from_records(composite_list) #creates dataframe to store all data
    df_Smarthome.columns = ['time','CO','Humidity','Light','lpg','motion','smoke','temp']
    df_Smarthome.to_csv('smarthome.csv', index = False)
    

In [4]:
# Defines callback function to print the subscribed message from the server for the mobile_device
data_store=[]

def on_message_mobile_device(client, userdata, msg):
    
    a=msg.payload.decode("utf-8")
    print(f"{msg.topic} {str(a)}")
    data_store.append(a)
    if msg.topic=="house/livingroom/temp":
        print('\n')
        update_df(data_store)
    

In [5]:
# Defines callback function to show when devices have connected. 

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

In [6]:
# Defines callback function to print the subscribed message from the server for the devices

def on_message(client, userdata, msg):
    
    a=msg.payload.decode("utf-8")
    print(f"{msg.topic} {str(a)}")
    print("")
 

## 2.0 MQTT Protocol Execution

* The following code will have clients subscribed to specific topics receive messages from the broker (i.e. commands or data) and print them 
* Mobile Device will receive sensor data messages and contoller devices will receive command data messages from mobile devices

In [7]:
# The portion of the code will focus on the controller commands from the mobile_device

if __name__ == '__main__':
   
    client8=mqtt.Client(client_id="temp_Controller") #Define Clients
    client9=mqtt.Client(client_id="light_Controller")
    client10=mqtt.Client(client_id="fan_Controller")
    
    client8.connect("localhost", 1883) #Connect client to server 
    client9.connect("localhost", 1883) 
    client10.connect("localhost", 1883) 
        
    client8.loop_start()  #Start client loop
    client8.subscribe("house/livingroom/temp/set")   
    client8.on_message = on_message #Print published message
   
    client9.loop_start()
    client9.subscribe("house/livingroom/Light")
    client9.on_message = on_message
        
    client10.loop_start()
    client10.subscribe("house/livingroom/Fan")
    client10.on_message = on_message

house/livingroom/temp/set 100house/livingroom/Light ON

house/livingroom/Fan ON





In [8]:
#The mobile_device will connect to the broker, recieve the published messages, and print them. 

if __name__ == '__main__':
    
    client_mobile_device = mqtt.Client()
    
    client_mobile_device.connect("localhost", 1883)
        
    client_mobile_device.loop_start()

    client_mobile_device.on_connect = on_connect_mobile_device
    client_mobile_device.on_message = on_message_mobile_device 
    
# wait = input("Press Enter to continue.") #Please press enter only once the Client_publisher.py has been executed

Connected with result code 0


## 4.0 Smarthome GUI 

In [9]:
# The portion of the code will create a GUI to display the results 
# i.e. the subcribed sensor data

window=Tk() #Load GUI screen
window.title('Smarthome MQTT Protocol Simulation')
window.geometry("400x310")
df_Smarthome=pd.read_csv("smarthome.csv")
lbl=Label(window, text="Smarthome Application", fg='Blue', font=("Helvetica", 16)) #Create title 
lbl.place(x=80, y=10)

# Create and place sensor ids as labels below

lbl1=Label(window, text= "Date / Time")
lbl1.place(x=20, y=70)
lbl2=Label(window, text= "CO Levels (ppm %)")
lbl2.place(x=20, y=100)
lbl3=Label(window, text= "Humidity Levels (%)" )
lbl3.place(x=20, y=130)
lbl4=Label(window, text= "Light Detected")
lbl4.place(x=20, y=160)
lbl5=Label(window, text= "LPG Levels (ppm %)")
lbl5.place(x=20, y=190)
lbl6=Label(window, text= "Motion Detected")
lbl6.place(x=20, y=220)
lbl7=Label(window, text= "Smoke Levels (ppm %)")
lbl7.place(x=20, y=250)
lbl8=Label(window, text= "Temperature (deg F)")
lbl8.place(x=20, y=280)

lbl9=Label(window, text="Connection Established",bg="green",fg="white")
lbl9.place(x=250, y= 40)

#Create and prints sensor data (i.e.e published/subscribed messages) as labels

#Sets variables to be called in a function
variable_Time=StringVar()
variable_CO=StringVar()
variable_Humidity=StringVar()
variable_Light=StringVar()
variable_LPG=StringVar()
variable_Motion=StringVar()
variable_Smoke=StringVar()
variable_Temp=StringVar()

def update_label(): #Creates function to update through sensor data
    for [ind,row] in df_Smarthome.iterrows(): #iterates through sensor dataframe
        variable_Time.set(str(row[0]))
        variable_CO.set(str(row[1]))
        variable_Humidity.set(str(row[2]))
        variable_Light.set(str(row[3]))
        variable_LPG.set(str(row[4]))
        variable_Motion.set(str(row[5]))
        variable_Smoke.set(str(row[6]))
        variable_Temp.set(str(row[7]))
        window.update()
        window.after(1000) 
        
lbl1data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable=variable_Time)
lbl1data.place(x=150,y=70)
lbl2data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable=variable_CO)
lbl2data.place(x=150,y=100)
lbl3data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable=variable_Humidity)
lbl3data.place(x=150,y=130)
lbl4data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable=variable_Light)
lbl4data.place(x=150,y=160)
lbl5data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable= variable_LPG)
lbl5data.place(x=150,y=190)
lbl6data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable=variable_Motion)
lbl6data.place(x=150,y=220)
lbl7data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable=variable_Smoke)
lbl7data.place(x=150,y=250)
lbl8data=Label(window, borderwidth=3, width=25, relief='sunken',textvariable=variable_Temp)
lbl8data.place(x=150,y=280)

start_button=Button(window,text="Start Simulation",command=update_label)
start_button.place(x=20,y=40)

window.mainloop()

house/livingroom/time 2020-07-12 00:01:34.385969877
house/livingroom/CO 0.004955939
house/livingroom/humidity 51.0
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007650822
house/livingroom/motiondetected No
house/livingroom/smoke 0.02041127
house/livingroom/temp 22.7


house/livingroom/time 2020-07-12 00:02:11.247800112
house/livingroom/CO 0.004956119
house/livingroom/humidity 50.9
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007651024
house/livingroom/motiondetected No
house/livingroom/smoke 0.020411845
house/livingroom/temp 22.6


house/livingroom/time 2020-07-12 00:02:48.364009857
house/livingroom/CO 0.004941779
house/livingroom/humidity 50.9
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007635003
house/livingroom/motiondetected No
house/livingroom/smoke 0.020366191
house/livingroom/temp 22.6


house/livingroom/time 2020-07-12 00:03:25.213350058
house/livingroom/CO 0.004984649
house/livingroom/humidity 51.0
house/livingroom/lightdetected Off
house

house/livingroom/time 2020-07-12 00:20:01.154890060
house/livingroom/CO 0.005033871
house/livingroom/humidity 50.5
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007737666
house/livingroom/motiondetected No
house/livingroom/smoke 0.02065885
house/livingroom/temp 22.3


house/livingroom/time 2020-07-12 00:20:37.939199924
house/livingroom/CO 0.005061692
house/livingroom/humidity 50.8
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007768579
house/livingroom/motiondetected No
house/livingroom/smoke 0.020747022
house/livingroom/temp 22.3


house/livingroom/time 2020-07-12 00:21:14.727630138
house/livingroom/CO 0.005073504
house/livingroom/humidity 50.6
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007781689
house/livingroom/motiondetected No
house/livingroom/smoke 0.020784424
house/livingroom/temp 22.3


house/livingroom/time 2020-07-12 00:21:51.585080147
house/livingroom/CO 0.005022228
house/livingroom/humidity 51.0
house/livingroom/lightdetected Off
house

house/livingroom/time 2020-07-12 00:38:27.664119959
house/livingroom/CO 0.005030915
house/livingroom/humidity 51.1
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007734379
house/livingroom/motiondetected No
house/livingroom/smoke 0.020649475
house/livingroom/temp 22.6


house/livingroom/time 2020-07-12 00:39:04.524889946
house/livingroom/CO 0.005023689
house/livingroom/humidity 51.1
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007726342
house/livingroom/motiondetected No
house/livingroom/smoke 0.020626555
house/livingroom/temp 22.6


house/livingroom/time 2020-07-12 00:39:41.384490013
house/livingroom/CO 0.005030957
house/livingroom/humidity 51.1
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007734426
house/livingroom/motiondetected No
house/livingroom/smoke 0.020649609
house/livingroom/temp 22.6


house/livingroom/time 2020-07-12 00:40:18.488440037
house/livingroom/CO 0.005051393
house/livingroom/humidity 51.1
house/livingroom/lightdetected Off
hous

house/livingroom/time 2020-07-12 00:56:53.928769827
house/livingroom/CO 0.005007697
house/livingroom/humidity 50.8
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007708541
house/livingroom/motiondetected No
house/livingroom/smoke 0.020575797
house/livingroom/temp 22.2


house/livingroom/time 2020-07-12 00:57:30.796839952
house/livingroom/CO 0.004980273
house/livingroom/humidity 50.7
house/livingroom/lightdetected Off
house/livingroom/lpg 0.00767798
house/livingroom/motiondetected No
house/livingroom/smoke 0.020488672
house/livingroom/temp 22.2


house/livingroom/time 2020-07-12 00:58:07.675729990
house/livingroom/CO 0.005000431
house/livingroom/humidity 51.2
house/livingroom/lightdetected Off
house/livingroom/lpg 0.007700449
house/livingroom/motiondetected No
house/livingroom/smoke 0.020552724
house/livingroom/temp 22.2


house/livingroom/time 2020-07-12 00:58:44.559210062
house/livingroom/CO 0.005003345
house/livingroom/humidity 51.4
house/livingroom/lightdetected Off
house

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\crazy\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-9-d08b8b7b4de2>", line 55, in update_label
    window.update()
  File "c:\users\crazy\appdata\local\programs\python\python38\lib\tkinter\__init__.py", line 1305, in update
    self.tk.call('update')
_tkinter.TclError: can't invoke "update" command: application has been destroyed
